In [2]:
import numpy as np
from Py6S import *
# Create a SixS object called s (used as the standard name by convention)

In [25]:
import os

In [32]:
output = {}
for p in params:
    s = SixS()
    s.ground_reflectance = GroundReflectance.HomogeneousLambertian(1)
    geom = Geometry.User()
    geom.from_time_and_location(lat=p[1],
                                lon=p[2],
                                datetimestring=p[0],
                                view_z=0, view_a=0)
    s.geometry = geom
    s.wavelength = Wavelength(0.550)
    s.altitudes.set_sensor_satellite_level()
#     s.write_input_file(os.path.join('atmospheres', p[3]))
    s.run
    s.outputs.values

In [37]:
SixSHelpers.Wavelengths.run_wavelengths(s, [.4,.5,.6], output_name='pixel_radiance')

Running for many wavelengths - this may take a long time


(array([0.4, 0.5, 0.6]), array([178.373, 261.821, 250.194]))

In [44]:
np.linspace(0.01, 1, 2)

array([0.01, 1.  ])

In [43]:
import json
with open('path', 'w') as f:
    f.write(json.dumps(s.outputs.values))

In [28]:
import pandas as pd
pd.DataFrame(output)

,a,b,c
version,1.1,1.1,1.1
month,3,1,6
day,1,3,22
solar_z,64,80,34
solar_a,181,182,186
view_z,0,0,0
view_a,0,0,0
scattering_angle,115.25,99.73,145.86
azimuthal_angle_difference,181.28,182.95,186.04
visibility,8.49,8.49,8.49


In [10]:
o.values

{'version': '1.1',
 'month': 6,
 'day': 22,
 'solar_z': 28,
 'solar_a': 180,
 'view_z': 0,
 'view_a': 0,
 'scattering_angle': 151.4,
 'azimuthal_angle_difference': 180.45,
 'visibility': 8.49,
 'aot550': 0.5,
 'ground_pressure': 1013.0,
 'ground_altitude': 0.0,
 'apparent_reflectance': 0.9914373,
 'apparent_radiance': 501.944,
 'total_gaseous_transmittance': 0.945,
 'wv_above_aerosol': 0.991,
 'wv_mixed_with_aerosol': 0.991,
 'wv_under_aerosol': 0.991,
 'percent_direct_solar_irradiance': 0.471,
 'percent_diffuse_solar_irradiance': 0.358,
 'percent_environmental_irradiance': 0.17,
 'atmospheric_intrinsic_reflectance': 0.07,
 'background_reflectance': 0.363,
 'pixel_reflectance': 0.558,
 'direct_solar_irradiance': 780.986,
 'diffuse_solar_irradiance': 593.926,
 'environmental_irradiance': 282.415,
 'atmospheric_intrinsic_radiance': 35.488,
 'background_radiance': 183.926,
 'pixel_radiance': 282.53,
 'solar_spectrum': 1811.651}

In [4]:
s.write_input_file('test.ini')

'test.ini'

In [14]:
wavelengths1, results1 = SixSHelpers.Wavelengths.run_wavelengths(s, np.arange(400,900,10)/1000,
                                                               output_name='pixel_radiance')

Running for many wavelengths - this may take a long time


In [1]:
from pyeosim import datasets
import xarray
import numpy as np
from Py6S import *
# Create a SixS object called s (used as the standard name by convention)
s = SixS()
s.ground_reflectance = GroundReflectance.HomogeneousLambertian(1)
geom = Geometry.User()
geom.from_time_and_location(lat=52.04, lon=0.76, datetimestring='2020/06/22T12:00',
                            view_z=0, view_a=0)
s.geometry = geom

s.altitudes.set_sensor_satellite_level()

# Run the 6S simulation defined by this SixS object across the
# whole VNIR range
wavelengths1, results1 = SixSHelpers.Wavelengths.run_wavelengths(s, np.arange(400,900,2)/1000,
                                                               output_name='pixel_radiance')
# Plot these results, with the y axis label set to "Pixel Radiance"
SixSHelpers.Wavelengths.plot_wavelengths(wavelengths1, results1, "Pixel Radiance")
ref_100 = xarray.DataArray(results1, [('wavelength', wavelengths1*1000)]).interp(
    wavelength = np.arange(400,900,1)
).interpolate_na('wavelength')

rho = np.arange(0,1,.001)
output = np.empty(shape=(len(rho), 500))

for i, _rho in enumerate(rho):
    _res = ref_100* _rho
    output[i,:] = _res

LUT = xarray.DataArray(output,[('rho', rho),('wavelength', np.arange(400,900,1))])
LUTd = LUT.to_dataset(name='pixel_radiance')
LUTd.attrs['6S_output_params'] = str(s.outputs.values)

ModuleNotFoundError: No module named 'Py6S'